<a href="https://colab.research.google.com/github/JoohoSon24/tire_competition/blob/main/%EC%86%90%EC%A3%BC%ED%98%B8_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp drive/MyDrive/25F/Tire/5-ai-and-datascience-competition.zip .
!mkdir -d data
!unzip 5-ai-and-datascience-competition.zip -d data/

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

from sklearn.preprocessing import StandardScaler, LabelBinarizer, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split

from imblearn.over_sampling import SMOTE

tab10_kwargs = {'cmap': 'tab10', 'vmin': 0, 'vmax': 9}

In [ ]:
def split_data(df, train=True):
    cols_sim = [f'{ch}{i}' for i in range(256) for ch in ['x', 'y', 'p']]
    X_sum = df.drop(columns=cols_sim)
    X_fem = df[cols_sim]
    if train:
      X_sum = X_sum.drop(columns='Class')
      y = df['Class']
      return X_sum, X_fem, y
    else:
      X_sum = X_sum.drop(columns='ID')
      ids = df['ID']
      return X_sum, X_fem, ids

def numerize(X_sum, oe=None):
  cat_cols = X_sum.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
  num_cols = X_sum.select_dtypes(exclude=['object', 'category', 'bool']).columns.tolist()

  train = oe is None

  Xn = X_sum[num_cols].values
  if train:
    oe = OneHotEncoder(sparse_output=False)
    Xc = oe.fit_transform(X_sum[cat_cols])
  else:
    Xc = oe.transform(X_sum[cat_cols])

  ohe_cols = oe.get_feature_names_out(cat_cols).tolist()
  X = np.concatenate([Xc, Xn], axis=1)
  X = pd.DataFrame(data=X, columns=ohe_cols+num_cols, index=X_sum.index)
  if train:
    return X, oe
  else:
    return X

def apply_smote(X, y, random_state=42):
    smote = SMOTE(random_state=random_state)
    X_res, y_res = smote.fit_resample(X, y)
    return X_res, y_res

In [ ]:
def curve_length(x, y, p):
    P = np.stack([x, y], axis=-1)
    return np.sqrt((np.diff(P, axis=-2) ** 2).sum(axis=-1)).sum(axis=-1)

def stress_length(x, y, p, p_thr=2.5):
    P = np.stack([x, y], axis=-1)  # (N, l, 2)
    mask = (p > p_thr)  # (N, l)

    ls = []
    for i in range(p.shape[0]):
        P_stress = P[i, mask[i], :]  # (l', 2)
        l = np.sqrt((np.diff(P_stress, axis=0) ** 2).sum(axis=-1)).sum(axis=-1)
        ls.append(l)
    ls = np.array(ls)

    return ls

def bend_extent(x, y, p):
    return (y.max(axis=-1) - y.min(axis=-1)) / (x.max(axis=-1) - x.min(axis=-1))

def max_curvature(x, y, p):
    dx  = np.gradient(x, axis=1)
    dy  = np.gradient(y, axis=1)
    ddx = np.gradient(dx, axis=1)
    ddy = np.gradient(dy, axis=1)

    num = np.abs(dx * ddy - dy * ddx)
    den = (dx**2 + dy**2)**1.5 + 1e-9

    curvature = num / den
    max_kappa = np.max(curvature, axis=1)
    return max_kappa

def FEM_feat(rows, fn=lambda x, y, p: p.max(axis=-1)):
    if isinstance(rows, (pd.Series, pd.DataFrame)):
        rows = rows.values
    x, y, p = np.transpose(rows.reshape(-1, rows.shape[1] // 3, 3), axes=(2, 0, 1))
    return fn(x, y, p)

def extract_fem(X_fem, fns=lambda x, y, p: p.max(axis=-1), as_df=False):
  if isinstance(X_fem, (pd.Series, pd.DataFrame)):
    rows = X_fem.values
  else:
    rows = X_fem

  if not isinstance(fns, list):
    fns = [fns]

  X_fem_new = []
  for fn in fns:
    feat = FEM_feat(X_fem, fn=fn)
    X_fem_new.append(feat)
  X_fem_new = np.stack(X_fem_new, axis=1)

  if as_df:
    X_fem_new = pd.DataFrame(data = X_fem_new, columns=[f'FEM_feat_{i}' for i in range(len(fns))])

  return X_fem_new

In [ ]:
def conf_table(y_pred, y_true):
    if y_true.ndim == 2:
      y_true = y_true.squeeze(1)
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    tp = ((y_pred == 1) & (y_true == 1)).sum()
    tn = ((y_pred == 0) & (y_true == 0)).sum()
    return tp, tn, fp, fn

def plot_curves(p_hat, y_true, ax, curve_type='auroc', n=100):
    if y_true.ndim == 2:
      y_true = y_true.squeeze(1)

    x, y = [], []
    for thr in np.linspace(0, 1, n + 1):
      y_pred = (p_hat > thr).astype(int)

      tp, tn, fp, fn = conf_table(y_pred, y_true)
      if curve_type == 'auroc':
        xx = fp / np.clip((tn + fp), 1, np.inf) # FPR
        yy = tp / np.clip((tp + fn), 1, np.inf) # Recall, TPR
      elif curve_type == 'auprc':
        xx = tp / np.clip((tp + fn), 1, np.inf) # Recall, TPR
        yy = tp / (tp + fp) if tp + fp > 0 else 1 # Precision
      elif curve_type == 'NPV':
        xx = thr
        yy = tn / (tn + fn) if tn + fn > 0 else 1
      elif curve_type == 'profit':
        xx = thr
        yy = tn * 100 - fn * 2000 - 99999 * int(fn + tn > 200 * len(y_true) / 466)
      x.append(xx)
      y.append(yy)

    x = np.array(x)
    y = np.array(y)

    if curve_type in ['auroc', 'auprc']:
      score = -((y[1:] + y[:-1]) / 2 * np.diff(x)).sum()
      ax.plot([0, 1], [0, 1] if curve_type == 'auroc' else [1, 0], 'k--', alpha=0.3)
      xlabel = 'FPR' if curve_type == 'auroc' else 'Recall'
      ylabel = 'TPR' if curve_type == 'auroc' else 'Precision'
      print(f"{curve_type.upper()} score: {score:.4f}")
    elif curve_type == 'NPV':
      score = y[len(y)//2]
      print(f"NPV at thr=0.5: {score:.4f}")
      xlabel = 'Threshold'
      ylabel = 'NPV'
    elif curve_type == 'profit':
      thr_opt = x[np.argmax(y)]
      score = np.max(y)
      print(f"Actual Optimal Profit: {score:.1f} at thr={thr_opt:.4f}")
      xlabel = 'Threshold'
      ylabel = 'Profit'

    ax.plot(x, y, c='red', lw=1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f'{curve_type} curve)')

    return score

def plot_profit(p_hat, y_true, ax, quantile=[0.25, 0.75], n=100, xlim=None, ylim=None):
    x, y_expected_mean, y_expected_std = [], [], []
    if y_true is None:
      y_true = np.zeros_like(p_hat)
    for thr in np.linspace(0, 1, n + 1):
      y_pred = (p_hat > thr).astype(int)
      p_selected = p_hat[p_hat < thr]

      y_bar = (100 * (1 - p_selected) - 2000 * p_selected).sum() - 99999 * int(len(p_selected) > 200 * len(p_hat) / 466)
      y_std = 2100 * np.sqrt((p_selected * (1 - p_selected)).sum())

      x.append(thr)
      y_expected_mean.append(y_bar)
      y_expected_std.append(y_std)

    x = np.array(x)
    y_expected_mean = np.array(y_expected_mean)
    y_expected_std = np.array(y_expected_std)

    #thr_opt_expected = x[np.argmax(y_expected_mean)]
    thr_opt_expected = 0.03
    p_max_expected = np.max(y_expected_mean)
    p_std_expected = y_expected_std[np.argmax(y_expected_mean)]

    y_pred = (p_hat > thr_opt_expected).astype(int)
    tp, tn, fp, fn = conf_table(y_pred, y_true)
    p_decision = tn * 100 - fn * 2000 - 99999 * int(fn + tn > 200 * len(y_true) / 466)

    print(f"Expected Optimal Profit: {p_max_expected:.1f}(±{p_std_expected:.1f}) at thr={thr_opt_expected:.4f}")
    print(f"Decision Profit: {p_decision:.1f} at thr={thr_opt_expected:.4f}")

    ax.plot(x, y_expected_mean, c='black', lw=1)
    y_q0 = y_q = y_expected_mean + norm.ppf(quantile[0]) * y_expected_std
    for q in quantile:
      y_q = y_expected_mean + norm.ppf(q) * y_expected_std
      ax.plot(x, y_q, c='grey', label=f"{q*100}%", lw=1)
    ax.fill_between(x, y_q0, y_q, fc='gray', alpha=0.3)

    if xlim is not None:
      ax.set_xlim(xlim)
    if ylim is not None:
      ax.set_ylim(ylim)

    ax.set_xlabel('Decision Threshold')
    ax.set_ylabel('Profit')
    ax.set_title(f'Profit curve')
    plt.legend()

    return thr_opt_expected, p_max_expected, p_std_expected, p_decision

def print_result(model, X, y_true):
    y_pred = model.predict(X)
    tp, tn, fp, fn = conf_table(y_pred, y_true)

    accuracy = (tp + tn) / (tp + fp + tn + fn)
    recall = (tp) / (tp + fn)
    precision = (tp) / (tp + fp)
    npv = (tn) / (tn + fn)
    f1 = 2 * (recall * precision) / (recall + precision)

    print(f"Ground truth positive rate: {y_true.mean():.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"NPV: {npv:.4f}")
    print(f"F1 score: {f1:.4f}")
    print()

def final_score(auroc, profit_mean, profit_std):
  return np.sqrt(max(auroc - 0.5, 0) / 0.5 * max(profit_mean, 0) / 20000).item()

In [ ]:
def plot_FEM(row, ax):
    if isinstance(row, pd.Series):
        row = row.values
    x, y, p = row.reshape(-1, 3).T
    im = ax.scatter(x, y, c=p, cmap='jet', s=0.5, vmin=1.2, vmax=3)
    plt.colorbar(im)

In [ ]:
import os
import shutil
from google.colab import files

# 1. 'data' 폴더 생성 (이미 있으면 무시)
# 주의: '/data'는 시스템 루트 경로이므로 권한 문제가 생길 수 있습니다. 현재 위치 기준인 'data'를 추천합니다.
if not os.path.exists('data'):
    os.makedirs('data')

print("아래 [파일 선택] 버튼을 눌러 train.csv와 test.csv를 모두 선택해 주세요.")

# 2. 로컬 파일 업로드 (train.csv, test.csv 선택)
uploaded = files.upload()

# 3. 업로드된 파일을 'data' 폴더로 이동
for filename in uploaded.keys():
    # 파일이 이미 data 폴더에 있다면 덮어쓰기 위해 이동 전 확인
    source = filename
    destination = os.path.join('data', filename)

    # 이동 (shutil.move는 파일 이동 명령어)
    shutil.move(source, destination)
    print(f"✅ {filename} 파일이 {destination} 위치로 이동되었습니다.")

In [ ]:
df_pub = pd.read_csv('data/train.csv')
df_exam = pd.read_csv('data/test.csv')

In [ ]:
X_sum, X_fem, cl_ = split_data(df_pub)
X_sum_exam, X_fem_exam, ids = split_data(df_exam, train=False)

le = LabelBinarizer()
cl = le.fit_transform(cl_)

In [ ]:
def cross_validate(model, X, y, cv=5, seed=None):
  if isinstance(y, pd.Series):
    y = y.values
  if y.ndim == 2:
    y = y.squeeze(1)

  N = X.shape[0]
  index = np.random.permutation(np.arange(N))
  X, y = X.iloc[index], y[index]

  block_size = N // cv
  ps, ys = [], []
  for i in range(cv):
    s, e = block_size * i, min(block_size * (i + 1), N)
    mask = ((np.arange(N) >= s) & (np.arange(N) < e))
    X_train, X_val = X.iloc[~mask], X.iloc[mask]
    y_train, y_val = y[~mask], y[mask]

    model.fit(X_train, y_train)
    p = model.predict_proba(X_val)[:, 1]
    ps.append(p)
    ys.append(y_val)
  ps = np.stack(ps)
  ys = np.stack(ys)

  return ps, ys

# 4. FEM Info. Extraction

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from tqdm.notebook import tqdm

def collate_fn(batch):
  keys = batch[0][-1].keys()
  xs = torch.stack([x for x, y, p, l in batch])
  ys = torch.stack([y for x, y, p, l in batch])
  ps = torch.stack([p for x, y, p, l in batch])
  labels = {k: torch.tensor([l[k] for x, y, p, l in batch]).to(torch.float32) for k in keys}
  return xs, ys, ps, labels

class FEMDataset(Dataset):
  def __init__(self, df, labels):
    self.df = df
    labels = (labels - labels.mean()) / labels.std()
    self.labels = {col: labels[col].values for col in labels.columns}

  def __len__(self):
    return len(self.df)

  def __getitem__(self, i):
    row = self.df.iloc[i].values
    x, y, p = row.reshape(-1, 3).T
    x = torch.from_numpy(x).to(torch.float32)
    y = torch.from_numpy(y).to(torch.float32)
    p = torch.from_numpy(p).to(torch.float32)

    x = (x - x[0]) / 100
    y = (y - y[0]) / 20

    l = {k: v[i] for k, v in self.labels.items()}
    return x, y, p, l

  def get_labels(self):
    return self.labels

class FEMEncoder(nn.Module):
  def __init__(self, in_dim=256*3, hid_dim=256, rep_dim=16, p=0.1, targets: list = ['G1', 'G2', 'G3']):
    super().__init__()
    self.body = nn.Sequential(
        nn.Linear(in_dim, hid_dim),
        nn.Dropout(p=p),
        nn.ReLU(),
        nn.Linear(hid_dim, rep_dim),
        nn.Dropout(p=p),
        nn.ReLU()
    )

    heads = {}
    for k in targets:
      heads[k] = nn.Linear(rep_dim, 1)
    self.heads = nn.ModuleDict(heads)
    self.targets = list(self.heads.keys())

  def get_repv(self, x, y, p):
    h = torch.cat([x, y, p], dim=-1)
    return self.body(h)

  def forward(self, x, y, p):
    h = self.get_repv(x, y, p)
    ys = {}
    for k in self.targets:
      ys[k] = self.heads[k](h).squeeze(-1)
    return ys

class CNNEncoder(FEMEncoder):
    def __init__(self, in_dim=3, hid_dim=64, rep_dim=16, n_blocks=4, p=.1, targets: list = ['G1', 'G2', 'G3']):
        """
        channels: 입력 채널 수 (x, y, p → 3)
        hid_dim: Conv hidden size
        layers: dilated residual block 개수
        """
        super().__init__(in_dim, hid_dim, rep_dim, p, targets)
        del self.body

        self.convs = nn.ModuleList()

        in_channels = in_dim
        out_channels = hid_dim
        dilation = 1
        for _ in range(n_blocks):
            padding = dilation
            block = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3,
                          dilation=dilation, padding=padding),
                nn.Dropout(p=p),
                nn.ReLU()
            )
            self.convs.append(block)
            in_channels = out_channels
            out_channels = hid_dim
            dilation *= 2

        self.mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, rep_dim)
        )

    def get_repv(self, x, y, p):
        """
        x, y, p: 각각 shape (batch, T)
        return: z (batch, hidden_dim)
        """
        # (B, T) → (B, 1, T)
        x = x.unsqueeze(1)
        y = y.unsqueeze(1)
        p = p.unsqueeze(1)

        # concat → (B, 3, T)
        h = torch.cat([x, y, p], dim=1)

        # CNN forward
        for conv in self.convs:
            h = conv(h)

        z = h.mean(dim=-1)

        # MLP → final representation
        z = self.mlp(z)
        return z

In [ ]:
def loss_medusa(preds, trues, weight=None):
  assert preds.keys() == trues.keys()

  loss = torch.zeros(1, device=list(preds.values())[0].device)
  for k in trues.keys():
    pred, true = preds[k], trues[k]
    assert pred.device == true.device, f"pred: {pred.device} / true: {true.device}"
    loss += nn.functional.mse_loss(pred, true)
  return loss

In [ ]:
# MODIFIED: Add Class label to DNN training targets
X_fem_all = pd.concat([X_fem, X_fem_exam], axis=0).reset_index(drop=True)
X_sum_all = pd.concat([X_sum, X_sum_exam], axis=0).reset_index(drop=True)

# Add Class column: train data has actual labels, exam data has dummy 0s
cl_all = pd.Series(
    np.concatenate([cl_.values, np.zeros(len(X_sum_exam))]),
    name='Class'
)
X_sum_all_with_class = pd.concat([X_sum_all, cl_all], axis=1)

N = len(X_fem_all)
r = .8
max_epoch = 100

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# MODIFIED: Add 'Class' to target_cols for DNN training
target_cols = ['Y1', 'G1', 'X1', 'Y5', 'X5', 'Y2', 'G2', 'G4', 'G3', 'Y3', 'Class']
loss_fn = loss_medusa

indices = torch.randperm(N)

train_set = FEMDataset(X_fem_all.iloc[indices[:int(N*r)]], labels=X_sum_all_with_class[target_cols].iloc[indices[:int(N*r)]])
val_set = FEMDataset(X_fem_all.iloc[indices[int(N*r):]], labels=X_sum_all_with_class[target_cols].iloc[indices[int(N*r):]])

train_loader = DataLoader(train_set, batch_size=16, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, collate_fn=collate_fn, shuffle=False)

In [ ]:
model_fem = CNNEncoder(hid_dim=256, targets=target_cols)
for batch in val_loader:
  x, y, p, ls = batch
  ls_pred = model_fem(x, y, p)
  print({k: v.shape for k, v in ls_pred.items()}); break

In [ ]:
model_fem = CNNEncoder(hid_dim=128, rep_dim=12, p=0.1, targets=target_cols)
optimizer = Adam(params=model_fem.parameters(), lr=1e-3, weight_decay=2e-4)

def run_epoch(train, epoch, device, verbose=True):
  loader = train_loader if train else val_loader
  epoch_loss = []
  bar = tqdm(loader) if verbose else loader
  for batch in bar:
    x, y, p, labels = batch
    x = x.to(device)
    y = y.to(device)
    p = p.to(device)
    for k in labels.keys():
      labels[k] = labels[k].to(device)

    labels_pred = model_fem(x, y, p)
    loss = loss_fn(labels_pred, labels)
    if train:
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    epoch_loss.append(loss.item())
  epoch_loss = sum(epoch_loss) / len(epoch_loss)
  return epoch_loss

model_size = sum([param.numel() for param in model_fem.parameters()])
print(f"Model size: {model_size}")

loss_dict = {'train': [], 'val': []}
model_fem.to(device)
for i in range(max_epoch):
  verbose = (i % 10 == 0)
  model_fem.train()
  train_loss = run_epoch(train=True, epoch=i, device=device, verbose=verbose)
  loss_dict['train'].append(train_loss)
  print(f"Epoch {i} training loss: {train_loss:.6f}") if verbose else None

  model_fem.eval()
  with torch.no_grad():
    val_loss = run_epoch(train=False, epoch=i, device=device, verbose=verbose)
  loss_dict['val'].append(val_loss)
  print(f"Epoch {i} validation loss: {val_loss:.6f}") if verbose else None

In [ ]:
labels = val_set.get_labels()
labels_pred = {k: [] for k in target_cols}
model_fem.eval()
for batch in val_loader:
  x, y, p, _ = batch
  x = x.to(device)
  y = y.to(device)
  p = p.to(device)
  with torch.no_grad():
    out = model_fem(x, y, p)
    for k in target_cols:
      labels_pred[k].append(out[k])

for k in target_cols:
  labels_pred[k] = torch.cat(labels_pred[k]).detach().cpu().numpy()

r, c = 6, 2
fig, axes = plt.subplots(r, c, figsize=(6 * c, 4 * r))

axes[0, 0].plot(loss_dict['train'], label='train')
axes[0, 0].plot(loss_dict['val'], label='val')
axes[0, 0].set_title("Loss plot")
axes[0, 0].legend()

for n, k in enumerate(target_cols):
  i, j = (n + 1) // c, (n + 1) % c
  ax = axes[i, j]
  label, label_pred = labels[k], labels_pred[k]
  m, M = np.sort(np.concatenate([label, label_pred]))[[0, -1]]

  ax.scatter(label, label_pred, s=5)
  ax.plot([m, M], [m, M], lw=0.5, c='black')
  ax.set_xlim(m - (M - m) * 0.05, M + (M - m) * 0.05)
  ax.set_ylim(m - (M - m) * 0.05, M + (M - m) * 0.05)

  SST = ((label - label.mean()) ** 2).sum()
  SSE = ((label_pred - label) ** 2).sum()
  R_sq =  1 - SSE / SST
  ax.set_title(f"Feature {k} - R sq.: {R_sq:4f}")

In [ ]:
# MODIFIED: Add Class to labels when extracting train FEM features
X_sum_with_class = pd.concat([X_sum, pd.Series(cl_.values, name='Class')], axis=1)
pub_set = FEMDataset(X_fem, labels=X_sum_with_class[target_cols])
pub_loader = DataLoader(pub_set, batch_size=16, collate_fn=collate_fn, shuffle=False)

rep_vs = []
model_fem.eval()
for batch in pub_loader:
  x, y, p, _ = batch
  x = x.to(device)
  y = y.to(device)
  p = p.to(device)
  with torch.no_grad():
    rep_vs.append(model_fem.get_repv(x, y, p))
rep_vs = torch.cat(rep_vs, dim=0).detach().cpu().numpy()

In [ ]:
df_fem_feat = pd.DataFrame(
    data = rep_vs,
    columns = [f'FEM_feat_{i}' for i in range(rep_vs.shape[1])]
)
df_fem_feat.to_csv('data/fem_ext.csv')
df_fem_feat

# 1. Model

In [ ]:
def engineer_features(X_sum):
    """
    Complete Feature Engineering Pipeline.
    Should be applied BEFORE numerize() to preserve categorical columns!

    Args:
        X_sum: DataFrame from split_data() with tabular features

    Returns:
        X_sum: DataFrame with engineered features
    """
    df = X_sum.copy()

    # === 1. Spec Engineering: 타이어 제원 기반 물리량 계산 ===
    # Sidewall Height (단면 높이 mm) = 단면폭 * 편평비 / 100
    df['Feat_Sidewall_H'] = df['Width'] * (df['Aspect'] / 100)

    # Total Diameter (타이어 전체 지름 mm) = (단면 높이 * 2) + (휠 인치 * 25.4)
    df['Feat_Diameter'] = (df['Feat_Sidewall_H'] * 2) + (df['Inch'] * 25.4)

    # Tire Volume Proxy (타이어 부피 근사치) = 지름 * 단면폭
    df['Feat_Volume_Proxy'] = df['Feat_Diameter'] * df['Width']

    # Aspect Ratio Interaction (인치 대비 폭 비율)
    df['Feat_Width_to_Inch'] = df['Width'] / (df['Inch'] + 1e-6)

    # === 2. Geometry Engineering: 형상 벡터 기하학적 처리 ===
    x_cols = [f'X{i}' for i in range(1, 6)]
    y_cols = [f'Y{i}' for i in range(1, 6)]

    if all(col in df.columns for col in x_cols + y_cols):
        # 원점으로부터의 거리
        dists = []
        for xc, yc in zip(x_cols, y_cols):
            dists.append(np.sqrt(df[xc]**2 + df[yc]**2))

        df['Feat_Geo_Mean_Dist'] = np.mean(dists, axis=0)
        df['Feat_Geo_Max_Dist'] = np.max(dists, axis=0)
        df['Feat_Geo_Std_Dist'] = np.std(dists, axis=0)

        # 형상 비대칭성
        x_range = df[x_cols].max(axis=1) - df[x_cols].min(axis=1)
        y_range = df[y_cols].max(axis=1) - df[y_cols].min(axis=1)
        df['Feat_Geo_Aspect_Ratio'] = x_range / (y_range + 1e-6)

        # Shoelace formula for polygon area
        area = 0
        for i in range(5):
            j = (i + 1) % 5
            term = df[f'X{i+1}'] * df[f'Y{j+1}'] - df[f'X{j+1}'] * df[f'Y{i+1}']
            area += term
        df['Feat_Geo_Area'] = np.abs(area) / 2.0

    # === 3. Process Param Engineering: 공정 파라미터 통계 ===
    proc_cols = [c for c in df.columns if 'Proc_Param' in c]
    if proc_cols:
        df['Feat_Proc_Mean'] = df[proc_cols].mean(axis=1)
        df['Feat_Proc_Std'] = df[proc_cols].std(axis=1)
        df['Feat_Proc_Max'] = df[proc_cols].max(axis=1)
        df['Feat_Proc_Min'] = df[proc_cols].min(axis=1)

    return df


print("✅ Feature Engineering 함수 정의 완료!")
print("   - engineer_features(X_sum) 함수를 numerize() 전에 호출하세요!")
print("   - Train/Test 모두 동일하게 적용해야 합니다!")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

from catboost import CatBoostClassifier
from xgboost import XGBClassifier, XGBRFClassifier

import shap

print("="*60)
print("TRAINING PIPELINE START")
print("="*60)

# Configuration - MODIFIED: Changed to CatBoost
model_type = 'catboost'  # 'catboost' 'xgb' 'rf'
include_fem = True
augment = False

# === STEP 1: Feature Engineering (BEFORE numerize!) ===
print("\n[1/4] Applying Feature Engineering...")
X_sum_engineered = engineer_features(X_sum)
print(f"   ✓ Original features: {X_sum.shape[1]}")
print(f"   ✓ After engineering: {X_sum_engineered.shape[1]}")

# === STEP 2: Model & Encoding ===
print("\n[2/4] Configuring model and encoding...")
if model_type == 'catboost':
    model = CatBoostClassifier(
        cat_features=['Plant', 'Proc_Param6'],
        verbose=0,
    )
    X = X_sum_engineered  # CatBoost handles categorical directly

elif model_type == 'xgb':
    model = XGBRFClassifier(
        n_estimators=1000,
        scale_pos_weight=1,
    )
    X, oe = numerize(X_sum_engineered)
    print(f"   ✓ Encoded features: {X.shape[1]}")

elif model_type == 'rf':
    model = RandomForestClassifier(
        n_estimators=1000,
        max_features="sqrt",
        bootstrap=True,
        criterion="log_loss"
    )
    X, oe = numerize(X_sum_engineered)
    print(f"   ✓ Encoded features: {X.shape[1]}")

print(f"   ✓ Model: {model_type.upper()}")

# === STEP 3: Add FEM features if needed ===
print("\n[3/4] Processing FEM features...")
if include_fem:
    X_fem_ext = df_fem_feat
    # IMPORTANT: Align indices before concat!
    X_fem_ext.index = X.index
    X = pd.concat([X, X_fem_ext], axis=1)
    print(f"   ✓ FEM features added. Total: {X.shape[1]} features")
else:
    print(f"   ✓ No FEM features. Total: {X.shape[1]} features")

print(f"\n✅ Pipeline complete! Final training data shape: {X.shape}")
print("="*60)

In [ ]:
if include_fem:
  X_fem_ext = df_fem_feat
  X = pd.concat([X, X_fem_ext], axis=1)

X_train, X_test, cl_train, cl_test = train_test_split(X, cl, train_size=0.8)
if augment:
  X_train, cl_train = apply_smote(X_train, cl_train)

In [ ]:
model.fit(X_train, cl_train)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap_value_pos = shap_values[:, :, 0]
shap.summary_plot(shap_value_pos, X_test, plot_type="dot", show=False)

In [ ]:
ps, ys = cross_validate(model, X, cl, cv=15)

In [ ]:
p = ps.flatten()
y = ys.flatten()
mask = (y == 0)

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

upper = 0.1
bins = np.linspace(0, upper, 11)
freq_n, _, im1 = axes[0].hist(p[mask], bins=bins, alpha=0.7)
freq_p, _, im2 = axes[0].hist(p[~mask], bins=bins, alpha=0.7)

acc_n = np.cumsum(freq_n)
acc_p = np.cumsum(freq_p)

p_pred = (bins[1:] + bins[:-1]) / 2

ratio = freq_p / (freq_n + freq_p)

bin_indices = np.digitize(p, bins)
bin_indices = np.clip(bin_indices, 1, len(bins) - 1) - 1
p_bin = ratio[bin_indices]

axes[1].plot(p_pred, ratio)
axes[1].scatter(p, p_bin, s=3, alpha=0.5, c=y, **tab10_kwargs)
axes[1].set_xlim(-upper * 0.1, upper * 1.1)
axes[1].set_ylim(-upper * 0.1, upper * 1.1)
axes[1].plot([0, upper], [0, upper], c='grey', lw=0.7, ls='--')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

auroc = plot_curves(p, y, axes[0, 0], curve_type='auroc', n=2000)
auprc = plot_curves(p, y, axes[0, 1], curve_type='auprc', n=2000)
npv = plot_curves(p, y, axes[1, 0], curve_type='NPV', n=2000)
p_actual = plot_curves(p, y, axes[1, 1], curve_type='profit', n=2000)
thr_opt, p_mean, p_std, p_dec = plot_profit(p, y, axes[1, 1], quantile=[0.05, 0.25, 0.75, 0.95], n=1000, xlim=(0, 0.1), ylim=(-10000, 21000))
print(f"final score (predicted): {final_score(auroc, p_mean, p_std):.4f}")
print(f"final score (actual upper bound): {final_score(auroc, p_actual, p_std):.4f}")
print(f"final score (decision): {final_score(auroc, p_dec, p_std):.4f}")
plt.tight_layout()

In [ ]:
# MODIFIED: Add Class dummy values for exam data
X_sum_exam_with_class = pd.concat([X_sum_exam, pd.Series(np.zeros(len(X_sum_exam)), name='Class')], axis=1)
pub_set_exam = FEMDataset(X_fem_exam, labels=X_sum_exam_with_class[target_cols])
pub_loader_exam = DataLoader(pub_set_exam, batch_size=16, collate_fn=collate_fn, shuffle=False)

rep_vs_exam = []
model_fem.eval()
for batch in pub_loader_exam:
  x, y, p, _ = batch
  x = x.to(device)
  y = y.to(device)
  p = p.to(device)
  with torch.no_grad():
    rep_vs_exam.append(model_fem.get_repv(x, y, p))
rep_vs_exam = torch.cat(rep_vs_exam, dim=0).detach().cpu().numpy()

df_fem_feat_exam = pd.DataFrame(
    data = rep_vs_exam,
    columns = [f'FEM_feat_{i}' for i in range(rep_vs.shape[1])]
)
df_fem_feat_exam.to_csv('data/fem_ext.csv')
df_fem_feat_exam

In [ ]:
print("="*60)
print("EXAM DATA PREDICTION & SUBMISSION")
print("="*60)

# === STEP 1: Retrain on full data ===
print("\n[1/5] Retraining model on full training data...")
model.fit(X, cl)
print("   ✓ Model retrained")

# === STEP 2: Apply EXACT SAME pipeline to exam data ===
print("\n[2/5] Processing exam data with same pipeline...")

# 2-1: Feature Engineering (BEFORE numerize!)
print("   - Applying Feature Engineering...")
X_sum_exam_engineered = engineer_features(X_sum_exam)
print(f"     ✓ Exam features: {X_sum_exam.shape[1]} → {X_sum_exam_engineered.shape[1]}")

# 2-2: Encoding with fitted encoder
print("   - Encoding categorical features...")
if model_type == 'catboost':
    X_exam = X_sum_exam_engineered
else:
    X_exam = numerize(X_sum_exam_engineered, oe=oe)  # Use fitted oe!
print(f"     ✓ Encoded features: {X_exam.shape[1]}")

# 2-3: Add FEM features
print("   - Adding FEM features...")
if include_fem:
    X_fem_ext_exam = df_fem_feat_exam
    # IMPORTANT: Align indices before concat!
    X_fem_ext_exam.index = X_exam.index
    X_exam = pd.concat([X_exam, X_fem_ext_exam], axis=1)
    print(f"     ✓ FEM features added. Total: {X_exam.shape[1]} features")

print(f"   ✓ Final exam data shape: {X_exam.shape}")

# === STEP 3: Verify column match ===
print("\n[3/5] Verifying column consistency...")
if X.shape[1] == X_exam.shape[1]:
    print(f"   ✅ Column count matches! Train: {X.shape[1]}, Exam: {X_exam.shape[1]}")
else:
    print(f"   ⚠️  WARNING: Column mismatch! Train: {X.shape[1]}, Exam: {X_exam.shape[1]}")
    print("   This will cause prediction errors!")

# === STEP 4: Predict ===
print("\n[4/5] Generating predictions...")
p_exam = model.predict_proba(X_exam)[:, 1]
print(f"   - Predictions generated: {len(p_exam)} samples")
print(f"   - Probability range: [{p_exam.min():.4f}, {p_exam.max():.4f}]")

# Profit curve analysis
print("\n   - Analyzing profit curve...")
ax = plt.subplot()
thr, p_mean, p_std, _ = plot_profit(
    p_hat=p_exam,
    y_true=None,
    ax=ax,
    n=1000,
    xlim=(0, 0.1),
    ylim=(-10000, 10000)
)
plt.show()

print(f"   ✓ Optimal threshold: {thr:.4f}")

print("\n" + "="*60)
print("PREDICTION COMPLETE")
print("="*60)

In [ ]:
import datetime
import os

print("="*60)
print("SUBMISSION FILE GENERATION")
print("="*60)

# === STEP 1: Use optimal threshold from profit curve ===
print(f"\n[1/3] Using optimal threshold from profit curve: {thr:.4f}")
decision = (p_exam < thr)
print(f"   ✓ Selected products: {decision.sum()} / {len(decision)}")
print(f"   ✓ Selection rate: {decision.sum()/len(decision)*100:.1f}%")

# === STEP 2: Load sample submission template ===
print("\n[2/3] Creating submission DataFrame...")
submission = pd.read_csv('data/sample_submission.csv')

# For L and R tires (466 each = 932 total), concatenate predictions
submission['probability'] = np.concatenate([p_exam, p_exam])
submission['decision'] = np.concatenate([decision, decision]).astype(int)

print(f"   ✓ Submission shape: {submission.shape}")
print(f"   ✓ Columns: {submission.columns.tolist()}")
print(f"   ✓ Total selected: {submission['decision'].sum()}")

# === STEP 3: Save submission file ===
print("\n[3/3] Saving submission file...")

# Create results folder if it doesn't exist
if not os.path.exists('results'):
    os.makedirs('results')
    print("   ✓ Created 'results/' folder")

# Generate filename with timestamp
now = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=9))).strftime("%m-%d-%H-%M")
filename = f"results/submission_HAIYONG_{now}.csv"

# Save without index
submission.to_csv(filename, index=False)

print(f"   ✓ File saved: {filename}")
print(f"\n📊 Submission Preview:")
print(submission.head(10))

print("\n" + "="*60)
print("✅ SUBMISSION FILE CREATED SUCCESSFULLY!")
print("="*60)
print(f"\n📁 Submit this file: {filename}")